In [7]:
import os
import sys

# Zakładamy, że nasz "utils.py" jest w katalogu nadrzędnym:
sys.path.append('..')

from utils import load_qa_pairs, evaluate_model, LanguageModel

# Inicjalizacja modelu
handler = LanguageModel()  # Zależnie od implementacji wewnątrz utils

[INFO] Loading model: eryk-mazus/polka-1.1b
[INFO] Using device: cuda:0


OutOfMemoryError: CUDA out of memory. Tried to allocate 344.00 MiB. GPU 0 has a total capacity of 9.78 GiB of which 2.31 MiB is free. Including non-PyTorch memory, this process has 9.78 GiB memory in use. Of the allocated memory 8.63 GiB is allocated by PyTorch, and 6.45 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import random

In [ ]:
def get_allowed_tokens_for_letter(letter: str, tokenizer):
    """
    Returns a list of all tokens in the tokenizer's vocabulary 
    that start with the given letter (or its uppercase version).
    
    :param letter: The letter we want tokens to start with (e.g. 'p').
    :param tokenizer: The tokenizer (with a .vocab or .get_vocab() method).
    :param lowercase: If True, also matches tokens that start with the uppercase version.
    :return: A list of tokens (strings) that match the criterion.
    """
    allowed_tokens = []
    allowed_ids = []
    
    # We assume 'tokenizer.get_vocab()' gives a dict {token_str: token_id}.
    vocab_dict = tokenizer.get_vocab()
    
    # Check both the given letter (e.g. 'p') and an uppercase version
    # if you want to allow 'P...' tokens as well.
    letter_upper = letter.upper()
    
    for tok_str, tok_id in vocab_dict.items():
        # Some tokens might have special characters or whitespace
        # We'll do a simple check that the first *alphabetic* character is our letter
        # but we can adapt or simplify as needed.
        
        if not tok_str:
            continue
        
        # If the very first character is letter or letter_upper
        # or if you want to skip special tokens like <s>, <pad>, etc.
        
        # Strip any leading special chars like 'Ġ' in some tokenizers
        # but this depends on the exact tokenizer you're using.
        # We'll do a naive approach for demonstration:
        
        clean_tok = tok_str.lstrip("Ġ")  # remove possible Byte-Level BPE leading
        if not clean_tok:
            continue
        
        first_char = clean_tok[0]
        second_char = clean_tok[1] if len(clean_tok) > 1 else ""
        
        # check if the first character is the letter we're looking for or "_"
        # if first_char != "▁" or second_char.lower() == letter.lower():
        #     allowed_tokens.append(tok_str)
        #     allowed_ids.append(tok_id)
        if first_char == "▁" or first_char.lower() == letter.lower():
            allowed_tokens.append(tok_str)
            allowed_ids.append(tok_id)
    
    return allowed_tokens, allowed_ids

# Example usage: let's see some tokens that start with 'p'
allowed_tokens_for_p, _ = get_allowed_tokens_for_letter('p', handler.tokenizer)
print(f"Number of tokens starting with 'p': {len(allowed_tokens_for_p)}")
print("Sample tokens (up to 30):", allowed_tokens_for_p[:30])
# save allowed tokens to file
# with open('allowed_tokens_for_p.txt', 'w') as f:
#     for token in allowed_tokens_for_p:
#         f.write(f'"{token}"' + '\n')

Number of tokens starting with 'p': 17891
Sample tokens (up to 30): ['▁storm', '▁akt', '▁своего', '▁lange', '▁aunque', '▁After', '▁proven', '▁Zwe', '▁del', '▁inhab', '▁band', '▁pkt', '▁Life', '▁chron', '▁Gemeins', '▁fasc', '▁om', '▁nada', '▁rescue', '▁revert', '▁Shakespeare', '▁Deb', '▁Kat', '▁Email', '▁ly', '▁verf', '▁cle', '▁adult', '▁story', '▁strik']


In [ ]:
# save all tokens from model dictionary to file
# with open('all_tokens.txt', 'w') as f:
#     for token in handler.tokenizer.get_vocab().keys():
#         f.write(f'"{token}"' + '\n')

In [ ]:
# Suppose we have some sentence prefixes from your example:
prefixes = [
    "Obowiązuje on od",
    "Został zrodzony ze",
    "Po pierwsze, projekt",
    "Po Panthers przejechali",
    "Duze dwusuwowe diesle",
    "Niestety, nikt nie",
    "Pani poseł, proszę",
    "Proszę państwa, po",
    "Proszę pana posła",
]

# We'll complete each prefix, forcing words to start with the letter 
# deduced from the last token's first letter or something similar.
# For demonstration, let's choose the last word from each prefix,
# take its first letter, build the allowed token list, and generate.

for pref in prefixes:
    # Extract the last word
    last_word = pref.strip().split()[-1]
    # We'll assume we want that last word's first letter
    letter = last_word[0].lower()  # e.g. "posła" -> 'p'
    
    # Build the allowed tokens
    allowed_tokens, allowed_ids = get_allowed_tokens_for_letter(letter, handler.tokenizer)
    # choose randomly 100 allowed_tokens
    allowed_tokens = random.sample(allowed_tokens, 10)
    
    # Generate text with constraints
    # We'll keep temperature=1.0 for variety, set max_new_tokens=15 for short completions
    completion = handler.generate_text_with_allowed_tokens(
        prompt=pref,
        allowed_ids=allowed_ids,
        allowed_tokens=allowed_tokens,
        max_new_tokens=15,
        temperature=0.3
    )
    
    # Print the result
    print("="*60)
    print(f"PREFIX: '{pref}'")
    print(", ".join([f"'{token}'" for token in allowed_tokens])) # allowed_tokens
    print(f"ALLOWED LETTER: '{letter}'")
    print(f"COMPLETION: {completion}")
    print("="*60)


PREFIX: 'Obowiązuje on od'
'▁Movie', '▁somehow', '▁реки', '▁nuovo', 'Other', '▁z▁w', '▁Prop', '▁miasta', '▁svo', '▁Pri'
ALLOWED LETTER: 'o'
COMPLETION: snuririririririririririririri
PREFIX: 'Został zrodzony ze'
'▁doch', '▁eigh', '▁heavily', '▁Види', '▁arbeit', '▁dov', '▁dispar', '▁functionality', '▁lookup', '▁zák'
ALLOWED LETTER: 'z'
COMPLETION: chavavchavavchavavchavavchavav
PREFIX: 'Po pierwsze, projekt'
'▁score', '▁grounds', '▁Luke', '▁gen', '▁nothing', 'proof', '▁zat', '▁rot', '▁мене', '▁degree'
ALLOWED LETTER: 'p'
COMPLETION: rotkekekekekekekekekekekekekeke
PREFIX: 'Po Panthers przejechali'
'▁Prem', '▁ad', '▁allocation', '▁statunitense', '▁dużo▁', '▁San', '▁Dit', '▁towards', '▁кораб', '▁witness'
ALLOWED LETTER: 'p'
COMPLETION: Dalit  Dalit  Dalit  Dal
PREFIX: 'Duze dwusuwowe diesle'
'▁сооб', '▁behaviour', '▁argued', '▁Mau', '▁recher', '▁GR', '▁htt', '▁Mrs', '▁indep', '▁interval'
ALLOWED LETTER: 'd'
COMPLETION: Minininininininininininininin
PREFIX: 'Niestety, nikt nie'
'▁well', '▁w

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.nn import functional as F
import random

model_name = 'eryk-mazus/polka-1.1b'  # Updated model name to match initialization

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

/pio/scratch/1/i317214/miniconda/envs/hallucination_detection/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
def best_k(prefix, K=10):
    input_ids = tokenizer(prefix, return_tensors='pt')['input_ids'].to(device)    
    output = model(input_ids=input_ids)
    next_token_logits = output.logits[0, -1, :]
    probs = F.softmax(next_token_logits, dim=-1)
    d = {}
    for i in range(probs.shape[0]):
        tok_str = tokenizer.decode(i).strip()
        # Remove leading special characters (e.g., 'Ġ' in some tokenizers)
        # tok_str_clean = tok_str.lstrip("Ġ")
        tok_str_clean = tok_str
        # Check if the token has an odd number of letters
        if (len(tok_str_clean) > 0 and tok_str_clean[0] != '▁') or \
            (len(tok_str_clean) > 1 and tok_str_clean[0] == '▁' and tok_str_clean[1] == 'p'):
            d[tok_str] = probs[i].item()
    
    # Sort tokens by probability in descending order and select top K
    sorted_tokens = sorted(d.items(), key=lambda x: x[1], reverse=True)
    return sorted_tokens[:K]

def sample_from_pairs(pairs): 
    tokens  = [p[0] for p in pairs]    
    weights = [p[1] for p in pairs]
    if not tokens:
        return ""  # Return empty string if no valid tokens
    return random.choices(tokens, weights=weights, k=1)[0]

# Example prefixes
start_txts = [
    "Pani poseł, proszę"
]

def sample_demo(N, txt):
    for i in range(N):
        d = best_k(txt)
        print(f"PREFIX: '{txt}'")
        if not d:
            print("   [INFO] No allowed tokens with odd number of letters found.")
            print("="*60)
            break
        next_token = sample_from_pairs(d)
        for t, p in d:
            star = '*' if t == next_token else ''
            print(f"   [{t}]{star} {p:.4f}")
        txt += next_token
        print("="*60)

# Run the demo for each prefix
for start_txt in start_txts:
    print(f"Generating completion for prefix: '{start_txt}'")
    sample_demo(30, start_txt)  # Generate 3 tokens for each prefix
    print("\n" + "#"*80 + "\n")


Generating completion for prefix: 'Pani poseł, proszę'
PREFIX: 'Pani poseł, proszę'
   [op] 0.0135
   [pamięta]* 0.0131
   [przekaz] 0.0081
   [nap] 0.0066
   [wys] 0.0051
   [poinform] 0.0050
   [,] 0.0050
   [zapyta] 0.0044
   [zab] 0.0031
   [się w] 0.0030
PREFIX: 'Pani poseł, proszępamięta'
   [ć, że] 0.0463
   [?] 0.0213
   [,]* 0.0166
   [, że] 0.0154
   [.] 0.0119
   [!] 0.0053
   [:] 0.0043
   [?\n] 0.0041
   [-] 0.0027
   [, co] 0.0019
PREFIX: 'Pani poseł, proszępamięta,'
   [może] 0.0058
   [op] 0.0028
   [mów] 0.0025
   [pon] 0.0021
   [mam] 0.0019
   [tr] 0.0018
   [pr] 0.0018
   [zn]* 0.0017
   [nap] 0.0017
   [pro] 0.0017
PREFIX: 'Pani poseł, proszępamięta,zn'
   [asz]* 0.4456
   [isz] 0.0442
   [ies] 0.0146
   [ika] 0.0079
   [osi] 0.0071
   [iew] 0.0070
   [ów z] 0.0026
   [ów w] 0.0023
   [ien] 0.0020
   [anego] 0.0020
PREFIX: 'Pani poseł, proszępamięta,znasz'
   [przepis] 0.0170
   [dobrze] 0.0124
   [,] 0.0098
   [?] 0.0089
   [prawo]* 0.0077
   [swoje] 0.0073
   [wy